<a href="https://colab.research.google.com/github/Fahrizaydr/Epub-Arabic-TTS/blob/main/Epub_to_Audio_Facebook_nllb_200_distilled_600M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Library

In [5]:
!pip install -q transformers datasets ebooklib beautifulsoup4 huggingface-hub

Upload Epub

In [2]:
from google.colab import files

# Upload file EPUB
uploaded = files.upload()

# Ambil nama file EPUB
epub_path = next(iter(uploaded))
print(f"File EPUB yang diupload: {epub_path}")


Saving معالم السنة النبوية - (1).epub to معالم السنة النبوية - (1).epub
File EPUB yang diupload: معالم السنة النبوية - (1).epub


Baca Epub

In [6]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re

def extract_text_from_epub(file_path):
    book = epub.read_epub(file_path)
    texts = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content()
            soup = BeautifulSoup(content, 'html.parser')
            texts.append(soup.get_text())

    full_text = "\n".join(texts)
    return full_text

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7Fء-ي]+', ' ', text)
    return text.strip()

# Ekstrak dan bersihkan teks
raw_text = extract_text_from_epub(epub_path)
arabic_text = clean_text(raw_text)

# Preview teks
print("Preview teks Arab:")
print(arabic_text[:1000])


Preview teks Arab:
الكتاب: معالم السنة النبوية المؤلف: صالح أحمد الشامي معالم السنة النبوية المجلد الأول (1/ 1) المصدر: الشاملة الذهبية الطبعة الثانية 1436 هـ ـ 2015 م (1/ 2) معالم السنة النبوية المجلد الأول (1/ 3) إهداء إلى كل مسلم ومسلمة أهدي هذه " المعالم ". قال ص ل ى الله  ع ل ي ه  و س ل م  ف ي حديث جبريل ع ل ي ه  الس ل ام  - كما عند ابن ماجه -: (ذاك جبريل أتاكم يعلمكم معالم دينكم). هذا  وأرجو الله تعالى - كما كان حديث جبريل بيان ا للمعالم الكلي ة للد ين - أن يكون هذا الكتاب بيانا للمعالم التفصيلية له. صالح أحمد الشامي (1/ 4) المقدمة إن الحمد لله نحمده  ونستعينه ونستغفره  ونعوذ بالله من شرور أنفسنا ومن سيئات أعمالنا  من يهد الله فلا مضل له  ومن يضلل فلا هادي له. وأشهد أن ل ا إ ل ه  إ ل ا الله  وحده لا شريك له  وأشهد أن م ح م د ا عبده ورسوله. أما بعد: فإن أشرف العلوم ما كان متصل ا بكتاب الله تعالى وس ن ة رسوله ص ل ى الله  ع ل ي ه  و س ل م  وأحمد الله تعالى أن يسر لي الخدمة في هذه الرحاب الكريم. فمنذ عشرين عام ا بدأت العمل على الجمع بين الصحيحين  ثم تبع ذلك كتب أخرى شكلت بمجموعها " م

Token Huggingface

In [7]:
from huggingface_hub import login
import getpass

token = getpass.getpass("Masukkan Hugging Face token kamu: ")
login(token)

Masukkan Hugging Face token kamu: ··········


Translate Arab > English

In [14]:
def split_text_by_period(text):
    sentences = text.split('.')
    chunks = []

    for sentence in sentences:
        clean_sentence = sentence.strip()
        if clean_sentence:
            chunks.append(clean_sentence + '.')

    return chunks

chunks = split_text_by_period(arabic_text)
print(f"Jumlah potongan teks: {len(chunks)}")


Jumlah potongan teks: 11083


In [ ]:
# 6. Load pipeline model NLLB dan terjemahkan
from transformers import pipeline

translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

def translate_text(text):
    return translator(text, src_lang="arb", tgt_lang="eng_Latn")[0]['translation_text']

translated_chunks = []

for idx, chunk in enumerate(chunks):
    print(f"🔁 Translating chunk {idx+1}/{len(chunks)}")
    try:
        translated = translate_text(chunk)
        translated_chunks.append(translated)
    except Exception as e:
        print(f"❌ Error on chunk {idx+1}: {e}")
        translated_chunks.append("")


Device set to use cpu


🔁 Translating chunk 1/11083
🔁 Translating chunk 2/11083
🔁 Translating chunk 3/11083
🔁 Translating chunk 4/11083
🔁 Translating chunk 5/11083
🔁 Translating chunk 6/11083


In [ ]:
translated_chunks = []

for idx, chunk in enumerate(chunks):
    print(f"🔄 Translating chunk {idx+1}/{len(chunks)}...")
    print(f"[Preview Chunk]: {chunk[:100]}...")
    translated_text = translate_via_api(chunk)
    translated_chunks.append(translated_text)
    time.sleep(1.5)  # Hindari rate limit

# Gabungkan hasil terjemahan
english_text = ' '.join(translated_chunks)

# Preview hasil terjemahan
print("\n✅ Preview hasil translate:")
print(english_text[:1000])


Streaming output truncated to the last 5000 lines.
[Preview Chunk]: ع ن أ ب ي م ر ث د ال غ ن و ي ق ال : ق ال ر س ول الله ص ل ى الله ع ل ي ه و س ل م : (ل ا ت ج ل س وا عل...
[!] Error 404: Not Found
🔄 Translating chunk 1464/3687...
[Preview Chunk]: الله ي و م ال ق ي ام ة ). [خ 1379 / م 2866] 36 - باب: سؤال القبر 1649 - (ق) ع ن ق ت اد ة ع ن أنس ب ن...
[!] Error 404: Not Found
🔄 Translating chunk 1465/3687...
[Preview Chunk]: م ن الن ار ق د أ ب د ل ك الله ب ه م ق ع د ا م ن ال ج ن ة ف ي ر اه م ا ج م يع ا). [خ 1374 (1338) / م ...
[!] Error 404: Not Found
🔄 Translating chunk 1466/3687...
[Preview Chunk]: ل س ن ا ح و ل ه ك أ ن م ا ع ل ى ر ء وس ن ا الط ي ر و ف ي ي د ه ع ود ي ن ك ت ب ه ف ي ال أ ر ض ف ر ف ع...
[!] Error 404: Not Found
🔄 Translating chunk 1467/3687...
[Preview Chunk]: د ين ي ال إ س ل ام ف ي ق ول ان ل ه : م ا ه ذ ا الر ج ل ال ذ ي ب ع ث ف يك م ) ق ال : (ف ي ق ول : ه و ...
[!] Error 404: Not Found
🔄 Translating chunk 1468/3687...
[Preview Chunk]: ن ة ) ق ال : (ف ي أ ت يه م ن ر و ح ه 

In [ ]:
# Membagi teks Inggris ke chunk
chunks_en = split_text(english_text, max_length=500)

# Translate Inggris ke Indonesia
translated_chunks_id = []
for idx, chunk in enumerate(chunks_en):
    print(f"Translating {idx+1}/{len(chunks_en)} (English -> Indonesian)...")
    translated_text = translate_via_api(chunk, src_lang="eng_Latn", tgt_lang="ind_Latn")
    translated_chunks_id.append(translated_text)
    time.sleep(1)

indonesian_text = ' '.join(translated_chunks_id)
print("\nPreview teks Indonesia:")
print(indonesian_text[:1000])


Translating 1/54 (English -> Indonesian)...
Error 400: {"error":"text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).","warnings":["There was an inference error: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples)."]}
Translating 2/54 (English -> Indonesian)...
Translating 3/54 (English -> Indonesian)...
Translating 4/54 (English -> Indonesian)...
Translating 5/54 (English -> Indonesian)...
Translating 6/54 (English -> Indonesian)...
Translating 7/54 (English -> Indonesian)...
Translating 8/54 (English -> Indonesian)...
Translating 9/54 (English -> Indonesian)...
Translating 10/54 (English -> Indonesian)...
Translating 11/54 (English -> Indonesian)...
Translating 12/54 (English -> Indonesian)...
Translating 13/54 (English -> Indonesian)...
Translating 14/54 (English -> Indonesian).

In [ ]:
from gtts import gTTS
import IPython.display as ipd

def text_to_speech(text, lang="in", filename="output.mp3"):
    tts = gTTS(text=text, lang=lang)
    tts.save(filename)
    return filename

#Hasil translate ke suara inggris
output_audio = text_to_speech(english_text, lang="en", filename="translated_audio.mp3")


gTTSError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [ ]:
#Hasil translate ke suara Indonesia
output_audio_id = text_to_speech(indonesian_text, lang="id", filename="translated_audio_id.mp3")

In [ ]:
# Memutar audio Inggris
print("Memutar hasil audio:")
ipd.display(ipd.Audio(output_audio))

# Memutar audio Indonesia
print("Memutar hasil audio Bahasa Indonesia:")
ipd.display(ipd.Audio(output_audio_id))


Memutar hasil audio:


NameError: name 'output_audio' is not defined

In [ ]:
# Simpan file teks Inggris
translated_file_en = "translated_text_en.txt"
with open(translated_file_en, "w", encoding="utf-8") as f:
    f.write(english_text)

# Simpan file teks Indonesia
translated_file_id = "translated_text_id.txt"
with open(translated_file_id, "w", encoding="utf-8") as f:
    f.write(indonesian_text)

from google.colab import files
# Download file Inggris
files.download(translated_file_en)
# Download file Indonesia
files.download(translated_file_id)
